## Skład grupy:

- **Jakub Anczyk**: funkcje obliczające
- **Piotr Gołąb**: przygotowanie i wczytanie danych, klasa reprezentująca Liczby Rozmyte, fuzyfikacja preferencji, Jupyter Notebook
- **Bartosz Sermak**: obliczenia, utworzenie struktury kodu, podział kodu na pliki, Jupyter Notebook


## Cel projektu
Celem projektu jest wykorzystanie metody *Fuzzy Analytical Hierarchy Process* (AHP) do podjęcia decyzji, które mieszkanie wybrać, na podstawie preferencji użytkownika.

## Dane
Dane opisujące mieszkania:
- `cena`
- `lokalizacja` - ocena lokalizacji, w której znajduje się mieszkanie (w skali od 1 do 10)
- `powierzchnia` - metraż mieszkania (w metrach kwadratowych)
- `wiek_budynku` - wiek budynku, w którym znajduje się mieszkanie (w latach)
- `bezpieczenstwo` - ocena bezpieczenstwa dzielnicy, w której znajduje się mieszkanie (w skali od 1 do 10)
- `odleglosc` - odleglosc mieszkania od centrum (w km)
- `infrastruktura` - ocena infrastruktury (dostępność komunikacji miejskiej, sklepów, restauracji, szkół, żłobków w okolicy) znajdującej się w pobliżu mieszkania (w skali od 1 do 10)

## Logika

Projekt jest zorganizowany w sposób modularny, co oznacza, że różne aspekty logiki aplikacji (przetwarzanie danych, obliczenia, konfiguracje) są oddzielone do różnych modułów.
Taka struktura umożliwia łatwe zarządzanie kodem, jego modyfikację i testowanie.
Centralizacja konfiguracji (`config.py`) oraz specjalistycznych funkcji (`data_utilities.py`, `calculations.py`) poprawia czytelność i utrzymanie kodu.
Główny plik `main.py` służy jako punkt wejścia do programu, integrując wszystkie moduły i orkiestrując przepływ procesów.


### Funkcje zostały podzielone na poszczególne pliki, co ułatwia poruszanie się po kodzie:

## 1. `fuzzy_number.py`
Ten plik zawiera definicję klasy `FuzzyNumber`, która reprezentuje rozmytą liczbę. Klasa ta zawiera metody umożliwiające wykonanie podstawowych operacji matematycznych na rozmytych liczbach, takich jak dodawanie, mnożenie, i potęgowanie, oraz metody do reprezentacji tekstowej liczby rozmytej (`__str__`, `__repr__`). Dodatkowo, zawiera metodę defuzzify, która służy do zdefuzyfikowania wartości rozmytej liczby.

In [6]:
class FuzzyNumber:
    def __init__(self, a: float, b: float = None, c: float = None):
        # Inicjalizacja rozmytej liczby w zależności od podanych wartości
        if b == None or c == None:
            if a == 1:
                self.a = 1
                self.b = 1
                self.c = 1
            elif a > 1:
                self.a = a - 1
                self.b = a
                self.c = a + 1
            elif a < 1 and a > 0:
                self.a = round(1 / ((1 / a) + 1), 2)
                self.b = round(a, 2)
                self.c = round(1 / ((1 / a) - 1), 2)
            else:
                raise Exception("Niepoprawne dane")
        elif a >= 0 and a <= b and b <= c:
            self.a = a
            self.b = b
            self.c = c
        else:
            raise Exception("Niepoprawne dane")


    # Dodawanie liczb rozmytych
    def __add__(self, other):
        return FuzzyNumber(self.a + other.a, self.b + other.b, self.c + other.c)

    # Mnożenie liczb rozmytych
    def __mul__(self, other):
        return FuzzyNumber(self.a * other.a, self.b * other.b, self.c * other.c)

    # Potęgowanie liczb rozmytych
    def __pow__(self, power):
        if power >= 0:
            return FuzzyNumber(self.a ** power, self.b ** power, self.c ** power)
        elif self.a > 0 and self.b > 0 and self.c > 0:
            return FuzzyNumber(1 / self.c ** abs(power), 1 / self.b ** abs(power), 1 / self.a ** abs(power))
        else:
            raise Exception("Niepoprawne dane")

    # Metody do reprezentacji tekstowej rozmytej liczby
    def __str__(self):
        return f'({self.a}, {self.b}, {self.c})'

    def __repr__(self):
        return f'({self.a}, {self.b}, {self.c})'

    def defuzzify(self):
        # Zdefuzyfikowanie rozmytej liczby
        return (self.a + self.b + self.c) / 3

## 2. `data_utilities.py`

W tym module znajdują się funkcje pomocnicze do wczytywania, przetwarzania i sprawdzania poprawności danych preferencji. Funkcje te obejmują:

- `fuzzify_preferences`: Konwertuje standardowe wartości liczbowe z DataFrame na rozmyte liczby (obiekty FuzzyNumber).
- `is_data_correct`: Sprawdza, czy dane w DataFrame są poprawne pod kątem wymogów analizy (np. czy wiersze i kolumny są zgodne, czy wartości są odwrotne).
- `read_preferences_data`: Wczytuje dane preferencji z pliku CSV i sprawdza ich poprawność.
- `check_if_columns_correct`: Sprawdza, czy kolumny danych zgadzają się z oczekiwanymi preferencjami.

In [7]:
import pandas as pd
import numpy as np
from models.fuzzy_number import FuzzyNumber

# Funkcja do konwersji preferencji na liczby rozmyte
def fuzzify_preferences(preferences: pd.DataFrame):
    fuzzy_preferences = list()

    for col_name in preferences.columns:
        pref_values = list()

        for row in preferences.columns:
            preference_value = preferences[col_name][row]
            fuzzy_value = FuzzyNumber(preference_value)
            pref_values.append(fuzzy_value)

        fuzzy_preferences.append(pref_values)

    return np.array(fuzzy_preferences, dtype=FuzzyNumber).transpose()

# Funkcja do sprawdzenia poprawności danych w DataFrame
def is_data_correct(data: pd.DataFrame):
    rows = sorted([row.lower() for row in data.index])
    cols = sorted([col.lower() for col in data.columns])

    if (rows != cols):
        return False

    # Sprawdzenie czy wartości poszczególnych preferencji są odwrotne
    for row in data.index:
        for col in data.columns:
            if (round(1 / data[row][col], 2) != round(data[col][row], 2)):
                return False

    return True

# Funkcja do wczytywania danych preferencji z pliku
def read_preferences_data(file_path: str):
    data = pd.read_csv(file_path, sep=";", index_col=0)

    if (is_data_correct(data)):
        return data
    else:
        raise Exception("Niepoprawne dane")

# Funkcja do sprawdzania, czy kolumny w danych zgadzają się z preferencjami
def check_if_columns_correct(preferences: list, data: pd.DataFrame):
    prefs = sorted([pref.lower() for pref in preferences])
    cols = sorted([col.lower() for col in data.columns])

    if (prefs != cols):
        raise Exception(f'Niepoprawne dane, kolumny preferencji: {prefs}, kolumny danych: {cols}')



## 3. `calculations.py`

Moduł ten zawiera funkcje odpowiedzialne za obliczenia matematyczne:

- `calculate_geometric_mean`: Oblicza średnią geometryczną danego wiersza danych.
- `calculate_geometric_means`: Oblicza średnie geometryczne dla wszystkich wierszy w zbiorze danych.
- `normalize`: Normalizuje daną tablicę wartości.
- `calculations`: Wykonuje główne obliczenia analityczne, łącząc powyższe funkcje, aby obliczyć wagi rozmyte i ostre dla danego zbioru preferencji.
- `is_consistent`: Sprawdza, czy podane preferencje są spójne, co jest kluczowe w procesie decyzyjnym opartym na metodzie AHP (Analytic Hierarchy Process).

In [8]:
import numpy as np


# Obliczanie średniej geometrycznej dla wiersza
def calculate_geometric_mean(row):
    num_columns = len(row)
    product = np.prod(row)
    geometric_mean = product ** (1 / num_columns)
    return geometric_mean

# Obliczanie średnich geometrycznych dla wszystkich wierszy
def calculate_geometric_means(preferences):
    geometric_means = np.empty(preferences.shape[0], dtype=object)

    for i, row in enumerate(preferences):
        geometric_mean = calculate_geometric_mean(row)
        geometric_means[i] = geometric_mean

    return geometric_means

# Normalizacja tablicy wartości
def normalize(arr):
    arrsum = np.sum(arr)
    norm_arr = np.zeros(arr.shape[0], dtype=float)

    for i, value in enumerate(arr):
        norm_arr[i] = value / arrsum

    return norm_arr

# Główna funkcja obliczeniowa
def calculations(preferences: np.array):
    geometric_means = calculate_geometric_means(preferences)
    geometric_means_sum = np.sum(geometric_means)
    fuzzy_weights = geometric_means * (geometric_means_sum ** -1)
    sharp_weights = np.empty(fuzzy_weights.shape[0], dtype=object)

    for i, value in enumerate(fuzzy_weights):
        sharp_weights[i] = value.defuzzify()

    norm_sharp_weights = normalize(sharp_weights)

    return (np.column_stack((fuzzy_weights, sharp_weights, norm_sharp_weights)))


# Sprawdzanie spójności preferencji
def is_consistent(pref, consistency_dict):
    pref_array = pref
    column_sums = np.sum(pref_array, axis=0)
    normalized_array = pref_array / column_sums
    row_mean = np.mean(normalized_array, axis=1)
    multiplied_array = pref_array * row_mean
    weighetmeans = np.sum(multiplied_array, axis=1)
    result = weighetmeans / row_mean
    consistency_index = (max(result) - len(result)) / (len(result) - 1)

    if consistency_index < consistency_dict[int(len(result))]:
        value = True
    else:
        value = False

    return value


## 4. `config.py`

W proponowanej strukturze katalogów służy jako centralne miejsce przechowywania konfiguracji i stałych, które są wykorzystywane w różnych miejscach projektu. Jego głównym celem jest umożliwienie łatwego dostępu do wspólnych ustawień i wartości, co sprzyja organizacji i utrzymaniu kodu.

In [9]:
import numpy as np

# Słownik do sprawdzania spójności
consistency_dict = {
    1: 0.00, 2: 0.00, 3: 0.58, 4: 0.90,
    5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41,
    9: 1.45, 10: 1.49
}

# Nazwy dla wyników obliczeń
names = np.array(("fuzzy_weight", "sharp_weight", "sharp_norm_weight"))


# Główna logika programu

## `main.py`

### Działanie

- Importowanie Modułów: main.py rozpoczyna się od importowania niezbędnych modułów i komponentów z innych plików projektu. W tym przypadku będą to moduły do wczytywania i przetwarzania danych (`data_utilities.py`), moduły obliczeniowe (`calculations.py`) oraz konfiguracyjne (`config.py`).

- Wczytywanie Danych: Następnie, main.py używa funkcji z `data_utilities.py` do wczytania danych preferencji z plików CSV. Dane te są wstępnie przetwarzane i sprawdzane pod kątem poprawności.

- Przetwarzanie Danych: Po wczytaniu danych, `main.py` przeprowadza ich przetwarzanie, w tym "rozmywanie" wartości preferencji (przekształcanie ich w liczby rozmyte) przy użyciu funkcji fuzzify_preferences.

- Obliczenia: Następnie, korzystając z funkcji z calculations.py, plik główny wykonuje różne obliczenia matematyczne, w tym obliczanie średnich geometrycznych, normalizację wyników oraz sprawdzanie spójności preferencji.

- Prezentacja Wyników: Na końcu, main.py prezentuje wyniki obliczeń, takie jak wagi rozmyte i ostre, w uporządkowanej formie.

In [10]:
import pandas as pd
from data import config as cfg
from data.data_utilities import read_preferences_data, fuzzify_preferences, check_if_columns_correct
from calculations.calculations import calculations, is_consistent

if __name__ == '__main__':

    # Importowanie danych preferencji, część I
    pref_data = read_preferences_data("./preferencje.csv")
    preferences = fuzzify_preferences(pref_data)

    # Importowanie danych preferencji, część II
    sub_pref_data = read_preferences_data("./preferencje-2.csv")
    sub_preferences = fuzzify_preferences(sub_pref_data)

    # Pobieranie nazw atrybutów z obu zestawów preferencji
    pref_columns = [col for col in pref_data.columns] + [col for col in sub_pref_data.columns]

    # Tworzenie tabli preferencji
    sharp_prefs_all = [pref_data, sub_pref_data]
    fuzzy_prefs_all = [preferences, sub_preferences]

    # Importowanie i sprawdzanie danych
    data = pd.read_csv("./dane.csv", sep=";", index_col=0)
    check_if_columns_correct(pref_columns, data)

    # Sprawdzanie spójności i przeprowadzenie obliczeń
    for i, value in enumerate(sharp_prefs_all):
        if is_consistent(value, cfg.consistency_dict):
            print("\nZestaw danych " + str(i + 1) + " jest spójny!\n")
            print("Poniżej znajduje się zestaw danych posortowany metodą AHP:\n\n")
            calculated_preferences = pd.DataFrame(calculations(fuzzy_prefs_all[i]),sharp_prefs_all[i].columns.transpose(), columns=cfg.names)
            selected_data = data[calculated_preferences.index]
            weighted_data = selected_data * calculated_preferences["sharp_norm_weight"]
            weighted_sum = weighted_data.sum(axis=1)
            weighted_sum = weighted_sum.apply(lambda x: round(x, 2))
            result_df = pd.DataFrame(selected_data)
            result_df['Weighted_Sum'] = weighted_sum
            result_df = result_df.sort_values(by='Weighted_Sum', ascending=False)
            print(result_df)
        else:
            print("\nNiestety, zestaw danych " + str(i + 1) + " nie jest spójny, dlatego preferencje nie zostały przeanalizowane.")


Zestaw danych 1 jest spójny!

Poniżej znajduje się zestaw danych posortowany metodą AHP:


                 Cena  Lokalizacja  Powierzchnia  Wiek_budynku  Weighted_Sum
mieszkanie_16  776667            7            35             2     481017.78
mieszkanie_6   776104            6            79             4     480671.57
mieszkanie_3   765566            1            42             3     474141.68
mieszkanie_11  712015            1            74             1     440977.54
mieszkanie_17  688605           10            38             8     426479.67
mieszkanie_19  683176            2            41            12     423115.98
mieszkanie_14  677537            8            78            11     419627.01
mieszkanie_15  660978            1            69             7     409368.99
mieszkanie_18  650914           10            32             9     403136.20
mieszkanie_7   642076            4            41             7     397661.49
mieszkanie_13  637120           10            57            1